In [1]:
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt

# Path of the file to read
fifa_filepath = "data.csv"
# Read the file into a variable iris_data
data = pd.read_csv(fifa_filepath)
# Print the first 5 rows of the data
data.head()


df2 = data.loc[:, 'Crossing':'Release Clause']
df1 = data[['Age', 'Overall', 'Value', 'Wage', 'Preferred Foot', 'Skill Moves', 'Position', 'Height', 'Weight']]
df = pd.concat([df1, df2], axis=1)

df = df.dropna()

def value_to_int(df_value):
    try:
        value = float(df_value[1:-1])
        suffix = df_value[-1:]

        if suffix == 'M':
            value = value * 1000000
        elif suffix == 'K':
            value = value * 1000
    except ValueError:
        value = 0
    return value
  
df['Value_float'] = df['Value'].apply(value_to_int)
df['Wage_float'] = df['Wage'].apply(value_to_int)
df['Release_Clause_float'] = df['Release Clause'].apply(lambda m: value_to_int(m))

def weight_to_int(df_weight):
    value = df_weight[:-3]
    return value
  
df['Weight_int'] = df['Weight'].apply(weight_to_int)
df['Weight_int'] = df['Weight_int'].apply(lambda x: int(x))

def height_to_int(df_height):
    try:
        feet = int(df_height[0])
        dlm = df_height[-2]

        if dlm == "'":
            height = round((feet * 12 + int(df_height[-1])) * 2.54, 0)
        elif dlm != "'":
            height = round((feet * 12 + int(df_height[-2:])) * 2.54, 0)
    except ValueError:
        height = 0
    return height

df['Height_int'] = df['Height'].apply(height_to_int)


df = df.drop(['Value', 'Wage', 'Release Clause', 'Weight', 'Height'], axis=1)

le_foot = preprocessing.LabelEncoder()
df["Preferred Foot"] = le_foot.fit_transform(df["Preferred Foot"].values)


for i in ['ST', 'CF', 'LF', 'LS', 'LW', 'RF', 'RS', 'RW']:
  df.loc[df.Position == i , 'Pos'] = 'Strikers' 

for i in ['CAM', 'CDM', 'LCM', 'CM', 'LAM', 'LDM', 'LM', 'RAM', 'RCM', 'RDM', 'RM']:
  df.loc[df.Position == i , 'Pos'] = 'Midfielder' 

for i in ['CB', 'LB', 'LCB', 'LWB', 'RB', 'RCB', 'RWB','GK']:
  df.loc[df.Position == i , 'Pos'] = 'Defender' 


from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score
lasso = linear_model.Lasso()
from sklearn.model_selection import train_test_split

le_class = preprocessing.LabelEncoder()

df['Pos'] = le_class.fit_transform(df['Pos'])

y = df["Pos"]

df.drop(columns=["Position","Pos"],inplace=True)


X_train, X_test, y_train, y_test = train_test_split(df, y, 
                                                    test_size=0.20, 
                                                    random_state=42 )
print(X_train.shape)
print(X_test.shape)

print(X_train.info())

(13314, 43)
(3329, 43)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 13314 entries, 12320 to 17334
Data columns (total 43 columns):
Age                     13314 non-null int64
Overall                 13314 non-null int64
Preferred Foot          13314 non-null int64
Skill Moves             13314 non-null float64
Crossing                13314 non-null float64
Finishing               13314 non-null float64
HeadingAccuracy         13314 non-null float64
ShortPassing            13314 non-null float64
Volleys                 13314 non-null float64
Dribbling               13314 non-null float64
Curve                   13314 non-null float64
FKAccuracy              13314 non-null float64
LongPassing             13314 non-null float64
BallControl             13314 non-null float64
Acceleration            13314 non-null float64
SprintSpeed             13314 non-null float64
Agility                 13314 non-null float64
Reactions               13314 non-null float64
Balance                 

In [ ]:

pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
                                    random_state=42, verbosity=2)
pipeline_optimizer.fit(X_train, y_train)
print(pipeline_optimizer.score(X_test, y_test))
pipeline_optimizer.export('tpot_exported_pipeline.py')


Generation 1 - Current best internal CV score: 0.8783982544500132
Generation 2 - Current best internal CV score: 0.8807276681912738
Generation 3 - Current best internal CV score: 0.8836569497759253
Generation 4 - Current best internal CV score: 0.8836569497759253
